In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 363.2 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: pip

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

In [ ]:
import os
from langchain_groq import ChatGroq
class GroqChatLLM(ChatGroq):
    def __init__(self, temperature=0, model_name="llama-3.1-70b-versatile", api_key=None):
        api_key = userdata.get("GROQ_API_KEY")

        if not api_key:
            raise ValueError("GROQ_API_KEY must be provided either as an argument or in the environment variables.")

        # Directly call the superclass (ChatGroq) initialization
        super().__init__(
            temperature=temperature,
            model_name=model_name,
            api_key=api_key
        )
######################################################################################################################
# from langchain_ollama import ChatOllama
# class OllamaLLM(ChatOllama):
    # def __init__(self, temperature=0, model_name="llama3.1"):
    #     # Directly call the superclass (ChatOllama) initialization with the fixed base URL
    #     super().__init__(
    #         temperature=temperature,
    #         model=model_name,
        # )

######################################################################################################################


In [ ]:
llm=GroqChatLLM()

In [ ]:
llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_time': 0.032, 'prompt_time': 0.01137101, 'queue_time': 3.060951193, 'total_time': 0.04337101}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-145fb324-79e9-415d-8d32-3287838e4353-0')

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
# from langsmith import LangsmithClient
import os

from google.colab import userdata
#set your Api keys
os.environ["SERPER_API_KEY"]=userdata.get("SERPER_API_KEY")#


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI
# from langsmith import LangsmithClient
import os
from google.colab import userdata
# Set up the Serper API key
os.environ["SERPER_API_KEY"] = "633c9f4f684e57cb300edf5a39a465f85f800c13"

# Create a Serper search tool
search_tool = SerperDevTool()

In [ ]:
pip install duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.9 MB/s eta 0:00:00


In [ ]:
"a71dffbb1968f78f3cf3e22f"

'a71dffbb1968f78f3cf3e22f'

In [ ]:

import requests
from bs4 import BeautifulSoup
# from dotenv import load_dotenv
# x = load_dotenv()
# print("Environment variables loaded successfully") if x else print("Error: Failed to load environment variables.")
###################################################################################################################
def calculate(expression: str) -> float:
    """
    Evaluate a mathematical expression and return the result as a float.
    :param expression: A string representing the mathematical expression to evaluate.
    :return: The result of the evaluation as a float. If an error occurs during evaluation, returns NaN.
    """
    try:
        print('-> TOOL-CALCULATE CALLED')
        return float(eval(expression))
    except Exception as e:
        # Return NaN (Not a Number) in case of an error
        return float('nan')
###################################################################################################################
import requests
def currency_converter(amount:str, from_currency: str, to_currency: str) -> str:
    """
    Convert an amount from one currency to another using real-time exchange rates.
    :param amount: The amount to convert in the source currency.
    :param from_currency: The currency to convert from.
    :param to_currency: The currency to convert to.
    :return: A string with the converted amount or an error message.
    """
    try:
        #api_key = os.environ.get('EXCHRATE_API_KEY')
        api_key = "a71dffbb1968f78f3cf3e22f"  # will expire on 11th Oct 2024
        # after expiry we can have free offer api key which have artes updates once per day.

        if not api_key:
            return "Error: API key not found. Set 'EXCHRATE_API_KEY' in environment variables."

        amount = float(amount)  # Convert amount to float

        # API URL with from_currency and to_currency
        url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for HTTP errors
        data = response.json()

        # Check for a successful response
        if data.get("result") != "success":
            return f"Error: Failed to retrieve exchange rate. {data.get('error-type', 'Unknown error')}"

        # Access the conversion rate
        rate = data['conversion_rate']
        converted = amount * rate

        return f"{converted:.2f}"

    except ValueError:
        return "Error: Invalid amount. Please provide a numeric value."
    except requests.RequestException as e:
        return f"Error fetching exchange rates: {str(e)}"
####################################################################################################################
import json
from duckduckgo_search import DDGS
from typing import Optional, Any
# Default headers for requests
DEFAULT_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# DuckDuckGo search function
def ddg_search(query: str, max_results = "10", headers: Optional[Any] = None, timeout: Optional[int] = 10) -> str:
    """
    Search DuckDuckGo for a query and return the results.

    :param query: The query to search for.
    :param max_results: The maximum number of results to return (default=10).
    :param headers: Optional headers for the request. If not provided, defaults to DEFAULT_HEADERS.
    :param timeout: Optional timeout for the request (default=10 seconds).
    :return: A JSON string containing the search results.
    """
    headers = headers or DEFAULT_HEADERS
    ddgs = DDGS(headers=headers, timeout=timeout)
    results = ddgs.text(keywords=query, max_results=int(max_results))
    return json.dumps(results, indent=2)
####################################################################################################################
# DuckDuckGo news function
def get_news(topic: str, max_results = "10") -> str:
    """
    Search DuckDuckGo for the latest news based on a query and return the results.
    :param topic: The query to search for news.
    :param max_results: The maximum number of news results to return (default=10).
    :return: A JSON string containing the news results.
    """
    headers = DEFAULT_HEADERS
    ddgs = DDGS(headers=headers, timeout=60)
    results = ddgs.news(keywords=topic, max_results=int(max_results))
    return json.dumps(results, indent=2)
####################################################################################################################
# Example Usage:
if __name__ == "__main__":
    import os
    #os.system('clear')
    # Example: Search for web results
    search_results = ddg_search("what daye is today in USA?")
    print("Search_results\n",14*'-')
    print(search_results)

    # Example: Search for news
    news_results = get_news("AI breakthroughs")
    print("News_results\n",14*'-')
    print(news_results)

    conversion =  currency_converter(100, 'USD', 'EUR')
    print("Conversion:\n",conversion)


Search_results
 --------------
[
  {
    "title": "Current Local Time in the United States - timeanddate.com",
    "href": "https://www.timeanddate.com/worldclock/usa",
    "body": "11:18 am. New York. EDT. UTC -4. See all Time Zones in United States. See Holidays in United States. Create a Calendar for United States. Time in States and Federal Districts in USA (51 States and Federal Districts Listed Below, 13 States and Federal Districts Have Multiple Time Zones) Alabama*."
  },
  {
    "title": "Calendar (What Day is Celebrated Today?) - Calendarr",
    "href": "https://www.calendarr.com/united-states/",
    "body": "Today is Tuesday, 15 October 2024. Day of the year: 289. Week of the year: 42. 2024: 79.0%. Today's Moon: Waxing Gibbous. October 2024 November 2024. Upcoming Holidays in the United States November. 11 Mon. Veterans Day. 28 Thu. Thanksgiving Day ... Select the date ; Day ..."
  },
  {
    "title": "Today's Date - CalendarDate.com",
    "href": "https://www.calendardate.c

In [ ]:
pip install crewai[tools]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: crewai-tools
    Found existing installation: crewai-tools 0.1.6
    Uninstalling crewai-tools-0.1.6:
      Successfully uninstalled crewai-tools-0.1.6


In [ ]:
from crewai_tools import BaseTool
import json
from duckduckgo_search import DDGS
import requests
import json
from duckduckgo_search import DDGS

class DdgSearchTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Performs a search on DuckDuckGo and returns results."

    def _run(self, query: str, max_results: str = "10", headers: dict = None, timeout: int = 10) -> str:
        """
        Search DuckDuckGo for a query and return the results.
        :param query: The query to search for.
        :param max_results: The maximum number of results to return (default=10).
        :param headers: Optional headers for the request. If not provided, defaults to DEFAULT_HEADERS.
        :param timeout: Optional timeout for the request (default=10 seconds).
        :return: A JSON string containing the search results.
        """
        headers = headers or {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        ddgs = DDGS(headers=headers, timeout=timeout)
        results = ddgs.text(keywords=query, max_results=int(max_results))
        return json.dumps(results, indent=2)

class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter Tool"
    description: str = "Converts an amount from one currency to another using real-time exchange rates."

    def _run(self, amount: str, from_currency: str, to_currency: str) -> str:
        """
        Convert an amount from one currency to another using real-time exchange rates.
        :param amount: The amount to convert in the source currency.
        :param from_currency: The currency to convert from.
        :param to_currency: The currency to convert to.
        :return: A string with the converted amount or an error message.
        """
        try:
            #api_key = os.environ.get('EXCHRATE_API_KEY')
            api_key = "a71dffbb1968f78f3cf3e22f" # will expire on 11th Oct 2024

            if not api_key:
                return "Error: API key not found. Set 'EXCHRATE_API_KEY' in environment variables."

            amount = float(amount)  # Convert amount to float

            # API URL with from_currency and to_currency
            url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"
            response = requests.get(url)
            response.raise_for_status()  # Raise exception for HTTP errors
            data = response.json()

            # Check for a successful response
            if data.get("result") != "success":
                return f"Error: Failed to retrieve exchange rate. {data.get('error-type', 'Unknown error')}"

            # Access the conversion rate
            rate = data['conversion_rate']
            converted = amount * rate

            return f"{converted:.2f}"

        except ValueError:
            return "Error: Invalid amount. Please provide a numeric value."
        except requests.RequestException as e:
            return f"Error fetching exchange rates: {str(e)}"
from crewai_tools import BaseTool
class CalculateTool(BaseTool):
    name: str = "Calculate Tool"   #tool name
    description: str = "Evaluates a mathematical expression and returns the result as a float."

    def _run(self, expression: str) -> float:
        """
        Evaluate a mathematical expression and return the result as a float.
        :param expression: A string representing the mathematical expression to evaluate.
        :return: The result of the evaluation as a float. If an error occurs during evaluation, returns NaN.
        """
        try:
            print('-> TOOL-CALCULATE CALLED')
            return float(eval(expression))
        except Exception as e:
            return float('nan')

class GetNewsTool(BaseTool):
    name: str = "Get News Tool"
    description: str = "Fetches the latest news based on a topic using DuckDuckGo."

    def _run(self, topic: str, max_results: str = "10") -> str:
        """
        Search DuckDuckGo for the latest news based on a query and return the results.
        :param topic: The query to search for news.
        :param max_results: The maximum number of news results to return (default=10).
        :return: A JSON string containing the news results.
        """
        headers = {"User-Agent": "Mozilla/5.0"}
        ddgs = DDGS(headers=headers, timeout=60)
        results = ddgs.news(keywords=topic, max_results=int(max_results))
        return json.dumps(results, indent=2)


In [ ]:
calculate_tool=CalculateTool()
currency_converter_tool=CurrencyConverterTool()
ddg_search_tool=DdgSearchTool()
get_news_tool=GetNewsTool()



In [ ]:
tools = [calculate_tool,currency_converter_tool, ddg_search_tool, get_news_tool]

In [ ]:
for i in range(len(tools)):
    print(tools[i])
    print("#"*50)

name='Calculate Tool' description="Calculate Tool(expression: 'string') - Evaluates a mathematical expression and returns the result as a float." args_schema=<class 'abc.CalculateToolSchema'> description_updated=False cache_function=<function BaseTool.<lambda> at 0x7d15f0f31870>
##################################################
name='Currency Converter Tool' description="Currency Converter Tool(amount: 'string', from_currency: 'string', to_currency: 'string') - Converts an amount from one currency to another using real-time exchange rates." args_schema=<class 'abc.CurrencyConverterToolSchema'> description_updated=False cache_function=<function BaseTool.<lambda> at 0x7d15f0f31870>
##################################################
name='DuckDuckGo Search Tool' description="DuckDuckGo Search Tool(query: 'string', max_results: 'string', headers: 'object', timeout: 'integer') - Performs a search on DuckDuckGo and returns results." args_schema=<class 'abc.DdgSearchToolSchema'> description_

In [ ]:
from crewai_tools import Tool  # Make sure you import the correct Agent class
from crewai import Agent, Task, Crew, Process
from jinja2 import Template



# # Assuming you have already initialized an LLM (e.g., ChatGroq, OpenAI)
llm = llm  # Replace with the actual LLM instance you are using



In [ ]:
# Defining the Jinja2 template for the system prompt
system_prompt_template = """

Question: {{ query }}

You are {{ role }}. Your goal is {{ goal }}.

Respond in this structured format:

Thought: Reason through the query logically, explaining the steps you'll take.
Action: tool_name: input arguments for the tool, based on the current step.
PAUSE

After observing the tool's output:

Thought: Analyze the observation and decide your next action.
Action: Specify the next tool or provide the final answer if all steps are complete.
Answer: Provide the final response.

{% if cot_example %}
Chain of Thought Example(s):
    {{ cot_example }}
{% endif %}

Begin!
"""

In [ ]:
from crewai import Task
from jinja2 import Template

# Assuming you have a researcher persona and system prompt template
calculator_persona = {
    "role": "mathematical assistant",
    "goal": "help users evaluate mathematical expressions accurately",
    "backstory": """You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information
     allows you to provide actionable insights, identify future trends, and assist researchers and professionals in
     navigating the rapidly evolving landscape of technology."""
}
calculator_agent= Agent(
    llm=llm,
    tools=[calculate_tool],  # Ensure tools are passed correctly
    role=calculator_persona["role"],
    goal=calculator_persona["goal"],  # Pass an actual topic
    backstory=calculator_persona["backstory"]  # Add the missing backstory
    , allow_delegation=False, allow_self_execution=True
)

# Example COT for the calculate task
calculate_cot_example = """
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 * 7 + 15 is 176.
"""

# Define the task using the Task object
calculate_task = Task(
    description=Template(system_prompt_template).render(
                role=calculator_persona["role"],
        goal=calculator_persona["goal"].format(query="{expression}"),

 query="{query}",
       cot_example=calculate_cot_example
    ),
    agent=calculator_agent,
    expected_output="""Answer the mathematical expression provided by the user."""
)


In [ ]:
print(calculator_agent)
print("#"*50)
calculate_task

formatting_errors=0 id=UUID('fe60bafb-3461-4106-be41-60ecf8ecee31') role='mathematical assistant' goal='help users evaluate mathematical expressions accurately' backstory='You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information\n     allows you to provide actionable insights, identify future trends, and assist researchers and professionals in\n     navigating the rapidly evolving landscape of technology.' cache=True config=None max_rpm=None verbose=False allow_delegation=False tools=[CalculateTool(name='Calculate Tool', description="Calculate Tool(expression: 'string') - Evaluates a mathematical expression and returns the result as a float.", args_schema=<class 'abc.CalculateToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7dde8814edd0>)] max_iter=25 max_execution_time=None agent_executor=CrewAgentExecutor(agent=RunnableAgent(runnable={
  input: RunnableLambda(...),
  tools: RunnableLambda(...),
  too

Task(description=

Question: {query}

You are mathematical assistant. Your goal is help users evaluate mathematical expressions accurately.

Respond in this structured format:

Thought: Reason through the query logically, explaining the steps you'll take.
Action: tool_name: input arguments for the tool, based on the current step.
PAUSE

After observing the tool's output:

Thought: Analyze the observation and decide your next action.
Action: Specify the next tool or provide the final answer if all steps are complete.
Answer: Provide the final response.


Chain of Thought Example(s):
    
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 * 7 + 15 is 176.



Begin!, expected_output=Answer the mathematical expression provide

In [ ]:

from crewai import Task
from jinja2 import Template

# currency_converter_persona=for currency conversion
currency_converter_persona = {
    "role": "currency conversion assistant",
    "goal": "help users convert currencies accurately using real-time exchange rates",
    "backstory": "You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology."
}
currency_converter_agent= Agent(
    llm=llm,
    tools=[currency_converter_tool],  # Ensure tools are passed correctly
    role=currency_converter_persona["role"],
    goal=currency_converter_persona["goal"],  # Pass an actual topic
    backstory=currency_converter_persona["backstory"]  # Add the missing backstory
    , allow_delegation=False, allow_self_execution=True
)


# Example COT for currency conversion task
currency_converter_cot_example = """
Question: What is USD 50 equivalent in EUR?
Thought: I need to convert 50 USD to EUR. I'll use the Currency Converter Tool for this.
Action: Currency Converter Tool: 50, "USD", "EUR"
PAUSE
Observation: 50 USD is equal to 42.15 EUR
Thought: The conversion is complete. I can now provide the answer.
Answer: 50 USD is equivalent to 42.15 EUR.
"""

# Define the task using the Task object
currency_converter_task = Task(
    description=Template(system_prompt_template).render(
        query="{query}",
                role=currency_converter_persona["role"],
goal=currency_converter_persona["goal"].format(query="{amount}, {from_currency}, {to_currency}"),
        cot_example=currency_converter_cot_example
    ),
    agent=currency_converter_agent,
    expected_output="""Answer the user's currency conversion request using real-time exchange rates."""
)


In [ ]:
print(currency_converter_agent)
print("#"*50)
currency_converter_task

formatting_errors=0 id=UUID('e0749ddb-940b-4ca3-83e1-8c36ad2bd0bd') role='currency conversion assistant' goal='help users convert currencies accurately using real-time exchange rates' backstory='You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology.' cache=True config=None max_rpm=None verbose=False allow_delegation=False tools=[CurrencyConverterTool(name='Currency Converter Tool', description="Currency Converter Tool(amount: 'string', from_currency: 'string', to_currency: 'string') - Converts an amount from one currency to another using real-time exchange rates.", args_schema=<class 'abc.CurrencyConverterToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7dde8814edd0>)] max_iter=25 max_execution_time=None agent_executor=CrewAgentExecuto

Task(description=

Question: {query}

You are currency conversion assistant. Your goal is help users convert currencies accurately using real-time exchange rates.

Respond in this structured format:

Thought: Reason through the query logically, explaining the steps you'll take.
Action: tool_name: input arguments for the tool, based on the current step.
PAUSE

After observing the tool's output:

Thought: Analyze the observation and decide your next action.
Action: Specify the next tool or provide the final answer if all steps are complete.
Answer: Provide the final response.


Chain of Thought Example(s):
    
Question: What is USD 50 equivalent in EUR?
Thought: I need to convert 50 USD to EUR. I'll use the Currency Converter Tool for this.
Action: Currency Converter Tool: 50, "USD", "EUR"
PAUSE
Observation: 50 USD is equal to 42.15 EUR
Thought: The conversion is complete. I can now provide the answer.
Answer: 50 USD is equivalent to 42.15 EUR.



Begin!, expected_output=Answer the user

In [ ]:
from crewai import Task
from jinja2 import Template

# searcher_persona for web search
researcher_persona = {
    "role": "web search assistant",
    "goal": "help users find information on Serper Search Tool",
    "backstory": "You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology."
}
researcher_agent= Agent(
    llm=llm,
    tools=[search_tool],  # Ensure tools are passed correctly
    role=researcher_persona["role"],
    goal=researcher_persona["goal"],  # Pass an actual topic
    backstory=researcher_persona["backstory"]  # Add the missing backstory
    , allow_delegation=False, allow_self_execution=True
)

# Example COT for DuckDuckGo search task
# ddg_search_cot_example = """
# Question: What are the main features of Python programming language?
# Thought: To answer this question, I need to search for information about Python's main features. I'll use the DuckDuckGo Search Tool.
# Action: "DuckDuckGo Search Tool: "main features of Python programming language"
# PAUSE
# Observation: The search results include:
# 1. Easy to learn and read
# 2. Interpreted language
# 3. Dynamically typed
# 4. Object-oriented programming support
# 5. Extensive standard library
# 6. Cross-platform compatibility
# Thought: I have the information needed to answer the question.
# Answer: The main features of Python programming language include: easy to learn and read syntax, interpreted execution, dynamic typing, strong support for object-oriented programming, an extensive standard library, and cross-platform compatibility.
# """
ddg_search_cot_example  = """
Thought: I need to find recent trends in {query}. I'll start with a broad search.
Action: Search the internet(search_query:"latest {query} developments, news, and trends" )
Observation: (Search results about recent {query} developments, news, and trends)
Thought: I've got some initial trends. Now I'll look for more specific developments.
Action: Search the internet(search_query: " breakthrough {query} technologies this year" )
Observation: (Search results about breakthrough technologies)
Thought: I have a comprehensive list now. I'll compile the top 15 trends.
Final Answer: Here's a list of 10 significant {query} trends and developments...
"""


# Define the task using the Task object
ddg_search_task = Task(
    description=Template(system_prompt_template).render(
                role=researcher_persona["role"],
        goal=researcher_persona["goal"].format(query="{query}"),
         query="{query}",
        cot_example=ddg_search_cot_example
    ),
    agent=researcher_agent,
    expected_output="""Provide the user with search results based on the query."""
)


In [ ]:
print(researcher_agent)
print("#"*50)
ddg_search_task


formatting_errors=0 id=UUID('5a2e61c2-07ac-4bbb-a226-dc03f0beb660') role='web search assistant' goal='help users find information on Serper Search Tool' backstory='You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology.' cache=True config=None max_rpm=None verbose=False allow_delegation=False tools=[SerperDevTool(name='Search the internet', description="Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.", args_schema=<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7dde8814edd0>, search_url='https://google.serper.dev/search', n_results=None)] max_iter=25 max_execution_time=None agent_executor=CrewAgentE

Task(description=

Question: {query}

You are web search assistant. Your goal is help users find information on Serper Search Tool.

Respond in this structured format:

Thought: Reason through the query logically, explaining the steps you'll take.
Action: tool_name: input arguments for the tool, based on the current step.
PAUSE

After observing the tool's output:

Thought: Analyze the observation and decide your next action.
Action: Specify the next tool or provide the final answer if all steps are complete.
Answer: Provide the final response.


Chain of Thought Example(s):
    
Thought: I need to find recent trends in {query}. I'll start with a broad search.
Action: Search the internet(search_query:"latest {query} developments, news, and trends" )
Observation: (Search results about recent {query} developments, news, and trends)
Thought: I've got some initial trends. Now I'll look for more specific developments.
Action: Search the internet(search_query: " breakthrough {query} technolog

In [ ]:
from crewai import Task
from jinja2 import Template

# news_persona persona for news search
news_persona = {
    "role": "news search assistant",
    "goal": "help users find the latest news on various topics",
    "backstory": "You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology."
}

news_agent= Agent(
    llm=llm,
    tools=[get_news_tool],  # Ensure tools are passed correctly
    role=news_persona["role"],
    goal=news_persona["goal"],  # Pass an actual topic
    backstory=news_persona["backstory"]  # Add the missing backstory
    , allow_delegation=False, allow_self_execution=True
)

# Example COT for Get News tool task
get_news_cot_example = """
Question: What are the latest developments in artificial intelligence?
Thought: To answer this question, I need to fetch recent news about artificial intelligence. I'll use the Get News Tool.
Action: Get News Tool: "artificial intelligence developments", "5"
PAUSE
Observation: The news results include:
1. "AI-powered drug discovery breakthrough announced by pharmaceutical company"
2. "New AI model achieves human-level performance in complex problem-solving tasks"
3. "Ethical concerns raised over AI's role in facial recognition technology"
4. "AI-driven climate modeling helps improve weather predictions"
5. "Tech giants collaborate on AI safety standards"
Thought: I have recent news about AI developments. I can now summarize the latest developments.
Answer: The latest developments in artificial intelligence include breakthroughs in drug discovery, advancements in problem-solving capabilities, ongoing discussions about ethical concerns, improvements in climate modeling, and collaborative efforts on AI safety standards.
"""

# Define the task using the Task object
get_news_task = Task(
    description=Template(system_prompt_template).render(
                role=news_persona["role"],
        goal=news_persona["goal"].format(query="{topic}, {max_results}"),
 query="{query}",
        cot_example=get_news_cot_example
    ),
    agent=news_agent,
    expected_output="""Provide the latest news articles and summaries based on the user's query."""
)


In [ ]:
print(news_agent)
print("#"*50)
get_news_task

formatting_errors=0 id=UUID('738d21b7-a813-45e2-9eca-4a48be4e98ec') role='news search assistant' goal='help users find the latest news on various topics' backstory='You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology.' cache=True config=None max_rpm=None verbose=False allow_delegation=False tools=[GetNewsTool(name='Get News Tool', description="Get News Tool(topic: 'string', max_results: 'string') - Fetches the latest news based on a topic using DuckDuckGo.", args_schema=<class 'abc.GetNewsToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7dde8814edd0>)] max_iter=25 max_execution_time=None agent_executor=CrewAgentExecutor(agent=RunnableAgent(runnable={
  input: RunnableLambda(...),
  tools: RunnableLambda(...),
  tool_names: RunnableLa

Task(description=

Question: {query}

You are news search assistant. Your goal is help users find the latest news on various topics.

Respond in this structured format:

Thought: Reason through the query logically, explaining the steps you'll take.
Action: tool_name: input arguments for the tool, based on the current step.
PAUSE

After observing the tool's output:

Thought: Analyze the observation and decide your next action.
Action: Specify the next tool or provide the final answer if all steps are complete.
Answer: Provide the final response.


Chain of Thought Example(s):
    
Question: What are the latest developments in artificial intelligence?
Thought: To answer this question, I need to fetch recent news about artificial intelligence. I'll use the Get News Tool.
Action: Get News Tool: "artificial intelligence developments", "5"
PAUSE
Observation: The news results include:
1. "AI-powered drug discovery breakthrough announced by pharmaceutical company"
2. "New AI model achieves hum

In [ ]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        # agents=[calculator_agent,currency_converter_agent, researcher_agent],
        agents=[researcher_agent],
        # tasks=[calculate_task,currency_converter_task, ddg_search_task],
        tasks=[ddg_search_task],
        verbose=True,
        process=Process.sequential
    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":
    # Run CrewAI for the topic 'AI and Data Science'
    result = run_crewai("What's are the top 10 latest news about AI ?")
    print(result)

 [DEBUG]: == Working Agent: web search assistant
 [INFO]: == Starting Task: 

Question: What's are the top 10 latest news about AI ?

You are web search assistant. Your goal is help users find information on Serper Search Tool.

Respond in this structured format:

Thought: Reason through the query logically, explaining the steps you'll take.
Action: tool_name: input arguments for the tool, based on the current step.
PAUSE

After observing the tool's output:

Thought: Analyze the observation and decide your next action.
Action: Specify the next tool or provide the final answer if all steps are complete.
Answer: Provide the final response.


Chain of Thought Example(s):
    
Thought: I need to find recent trends in What's are the top 10 latest news about AI ?. I'll start with a broad search.
Action: Search the internet(search_query:"latest What's are the top 10 latest news about AI ? developments, news, and trends" )
Observation: (Search results about recent What's are the top 10 latest 

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result))

Search results: 

Title: This AI Tool Helped Convict People of Murder. Then ... - WIRED
Link: https://www.wired.com/story/cybercheck-crime-reports-prosecutions/
Snippet: Global Intelligence claims its Cybercheck technology can help cops find key evidence to nail a case. But a WIRED investigation reveals the ...
---
Title: Best AI Search Engines To Try Right Now
Link: https://www.searchenginejournal.com/ai-search-engines/497061/
Snippet: The Best AI Search Engines To Try Right Now · 1. Brave AI Search · 2. Andi Search · 3. Perplexity AI · 4. Phind · 5. YOU AI Search Engine.
---
Title: What's the best web source for getting the most up date AI news?
Link: https://www.reddit.com/r/ArtificialInteligence/comments/1326nc0/whats_the_best_web_source_for_getting_the_most_up/
Snippet: AI News: This website provides the latest news and trends in artificial intelligence, including research breakthroughs, industry developments, ...
---
Title: Top 10 Artificial Intelligence (AI) SEO Tools in 2024 - Yahoo Finance
Link: https://finance.yahoo.com/news/top-10-artificial-intelligence-ai-200519447.html
Snippet: In this article, we will take a look at the top 10 artificial intelligence (AI) SEO tools in 2024.
---
Title: The best AI search engines of 2024: Google, Perplexity, and more
Link: https://www.zdnet.com/article/best-ai-search-engine/
Snippet: To help you decide which AI search engine is worth using, ZDNET tested the best tools we could find and rounded them up, with notes on key specs and hands-on ...
---
Title: AI News & Artificial Intelligence - TechCrunch
Link: https://techcrunch.com/category/artificial-intelligence/
Snippet: Read the latest on artificial intelligence and machine learning tech, the companies that are building them, and the ethical issues AI raises today.
---
Title: The Best AI Search Engines to Try in 2024 - HubSpot Blog
Link: https://blog.hubspot.com/marketing/ai-search-engines
Snippet: From Perplexity and Google Gemini to Brave Search and Komo — here's what to know about the leading AI search engines and how you can use ...
---
Title: 10 AI Tools You Can Start Using For Free | Google Cloud
Link: https://cloud.google.com/use-cases/free-ai-tools
Snippet: Google Cloud offers free usage of many AI products up to monthly limits, including Translation, Speech-to-Text, Natural Language, and Video Intelligence.
---
Title: CrewAI Google Search and Scraping Tools Made EASY! - YouTube
Link: https://www.youtube.com/watch?v=BAY71UzrT78
Snippet: In this video, I will show you how to use serper dev and web scraping tools for ai news ...
---
Title: NewsGuard Special Reports
Link: https://www.newsguardtech.com/reports/
Snippet: AI Chatbots Are Blocked by 67% of Top News Sites, Relying Instead on Low-Quality Sources. Most high-quality news sites request to block AI chatbots.

In [ ]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        agents=[calculator_agent],
        tasks=[calculate_task],
        verbose=True,
        process=Process.sequential
    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":

    result = run_crewai("what is 10*25-25")

 [DEBUG]: == Working Agent: mathematical assistant
 [INFO]: == Starting Task: 

Question: what is 10*25-25

You are mathematical assistant. Your goal is help users evaluate mathematical expressions accurately.

Respond in this structured format:

Thought: Reason through the query logically, explaining the steps you'll take.
Action: tool_name: input arguments for the tool, based on the current step.
PAUSE

After observing the tool's output:

Thought: Analyze the observation and decide your next action.
Action: Specify the next tool or provide the final answer if all steps are complete.
Answer: Provide the final response.


Chain of Thought Example(s):
    
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 * 7 + 15 is 176.

In [ ]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        agents=[calculator_agent,currency_converter_agent, researcher_agent],
        # agents=[currency_converter_agent],
        tasks=[calculate_task,currency_converter_task, ddg_search_task],
        # tasks=[currency_converter_task],
        verbose=True,
        process=Process.sequential,
        manager_llm=llm
    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":
    # Run CrewAI for the topic 'AI and Data Science'
    result = run_crewai("What is USD 29 equivalent in GBP")

 [DEBUG]: == Working Agent: mathematical assistant
 [INFO]: == Starting Task: 

Question: What is USD 29 equivalent in GBP

You are mathematical assistant. Your goal is help users evaluate mathematical expressions accurately.

Respond in this structured format:

Thought: Reason through the query logically, explaining the steps you'll take.
Action: tool_name: input arguments for the tool, based on the current step.
PAUSE

After observing the tool's output:

Thought: Analyze the observation and decide your next action.
Action: Specify the next tool or provide the final answer if all steps are complete.
Answer: Provide the final response.


Chain of Thought Example(s):
    
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 

KeyboardInterrupt: 

In [ ]:
Crew?

In [ ]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        agents=[calculator_agent,currency_converter_agent, researcher_agent],
        # agents=[researcher_agent],
        tasks=[calculate_task,currency_converter_task, ddg_search_task],
        # tasks=[ddg_search_task],
        verbose=True,
        process=Process.sequential
    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":
    # Run CrewAI for the topic 'AI and Data Science'
    result = run_crewai("What's are the top 10 latest news about AI ?")
    print(result)